In [16]:
import os
import time
from math import floor, ceil

import numpy
from keras import layers, metrics
from keras import models, optimizers
from keras.applications import VGG16
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from utils import *
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

start = time.time()

train_dir = '/home/snowflake/Documents/nowe_trendy/train'
test_dir = '/home/snowflake/Documents/nowe_trendy/test'
validation_dir = '/home/snowflake/Documents/nowe_trendy/validation'

path, dirs, files = os.walk(train_dir).__next__()

val_size = 0.15
nTrain = sum([len(files) for r, d, files in os.walk(train_dir)]) - len(dirs)
nVal = ceil(nTrain * val_size)
path, dirs, files = os.walk(test_dir).__next__()
nTest = sum([len(files) for r, d, files in os.walk(test_dir)]) - len(dirs)
nClasses = len(dirs)
batch_size = 20
dense_size = 256
epochs = 10

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

for layer in conv_base.layers[:-4]:
    layer.trainable = False
    
for layer in conv_base.layers:
    print(layer, layer.trainable)

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(dense_size, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(nClasses, activation='softmax'))

<keras.engine.topology.InputLayer object at 0x7f866bb3df28> False
<keras.layers.convolutional.Conv2D object at 0x7f866d7f7748> False
<keras.layers.convolutional.Conv2D object at 0x7f866d7f74a8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f86680f3a20> False
<keras.layers.convolutional.Conv2D object at 0x7f866d7e8748> False
<keras.layers.convolutional.Conv2D object at 0x7f86681000b8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f866c4444a8> False
<keras.layers.convolutional.Conv2D object at 0x7f8668107be0> False
<keras.layers.convolutional.Conv2D object at 0x7f866bb3f9e8> False
<keras.layers.convolutional.Conv2D object at 0x7f866e7d2f28> False
<keras.layers.pooling.MaxPooling2D object at 0x7f866e7e5a58> False
<keras.layers.convolutional.Conv2D object at 0x7f866bb90f60> False
<keras.layers.convolutional.Conv2D object at 0x7f866bb90e80> False
<keras.layers.convolutional.Conv2D object at 0x7f866bb9d400> False
<keras.layers.pooling.MaxPooling2D object at 0x7f866bb95908> Fa

In [17]:
epochs = 20
train_batchsize = 12
val_batchsize = 8
dense_size = 256

In [18]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 7875 images belonging to 12 classes.
Found 1818 images belonging to 12 classes.


In [19]:
print("**********Configure model for training************")


model.compile(optimizer=optimizers.RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=[metrics.categorical_accuracy])

**********Configure model for training************


In [20]:
print("**********Start of training************")
history = model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples/train_generator.batch_size,
                    epochs=epochs,
#                     batch_size=batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples/validation_generator.batch_size)

**********Start of training************
Epoch 1/20
 39/656 [>.............................] - ETA: 16:19 - loss: 2.7201 - categorical_accuracy: 0.1474

KeyboardInterrupt: 

In [ ]:
# model.save('fine_tuning.model')

In [ ]:
# Create a generator for prediction
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = '/home/snowflake/Documents/nowe_trendy/test'
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = test_generator.filenames
 
# Get the ground truth from generator
ground_truth = test_generator.classes
 
# Get the label to class mapping from the generator
label2index = test_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)

In [ ]:
errors = np.where(predicted_classes.round() != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),test_generator.samples))
accuracy = accuracy_score(ground_truth, predicted_classes)
print("Accuracy: {}".format(accuracy))

In [ ]:
# nTest = 1821

In [ ]:
# print("**********Getting test features************")
# test_features, test_labels = get_features_and_labels(nTest,
#                                                                      nClasses,
#                                                                      batch_size,
#                                                                      test_dir,
#                                                                      conv_base,
#                                                                     aug = False)

In [ ]:
# print("**********End of training************\n\n")

# ground_truth = [np.where(r == 1)[0][0] for r in test_labels]

# predictions = model.predict_classes(test_features)
# prob = model.predict(test_features)

# # errors = nVal - accuracy_score(ground_truth, predictions, normalize=False)
# errors = np.where(predictions != ground_truth)[0]
# accuracy = accuracy_score(ground_truth, predictions)
# precision, recall, f_score, support = score(ground_truth, predictions, average='weighted')

# print("No of errors = {}/{}".format(len(errors), nTest))
# print("Accuracy: {}".format(accuracy))
# print("Weighted average Precision: {}".format(precision))
# print("Weighted average Recall: {}".format(recall))
# print("Weighted average F-score: {}".format(f_score))

# end = time.time()
# print("Time: " + str(end - start))


In [ ]:
# f = open("/media/snowflake/Data/features.dat", "wb+")
# np.save(f, all_features)

In [ ]:
# f = open("/media/snowflake/Data/labels.dat", "wb+")
# np.save(f, all_labels)